In [12]:
import pandas as pd
import sqlite3
import yfinance as yf
from binance.client import Client
from sqlalchemy import create_engine

#### 1. Executing SQL with Python

In [3]:
conn = sqlite3.connect("data/test1.db") # after execution, test1.db is created

In [4]:
cursor = conn.cursor() # with this cursor object, we can execute sql statement

In [ ]:
"""
SQLite Datatypes:
- INTEGER
- REAL
- TEXT
- BLOBS (Binary large objects)
- NULL
"""
# create table requires assigning datatypes
cursor.execute("CREATE TABLE employees (name TEXT, surname TEXT, salary REAL)")
conn.commit() # nothing in db if don't commit

In [7]:
# insert rows
cursor.execute("INSERT INTO employees VALUES ('Maria','Lay',100000)")
conn.commit()

In [8]:
cursor.execute("INSERT INTO employees VALUES ('Lax','Carl',50000)")
conn.commit()

In [9]:
# query data from table
cursor.execute("Select * from employees")
cursor.fetchall()

[('Maria', 'Lay', 100000.0),
 ('Lax', 'Carl', 50000.0),
 ('Maria', 'Lay', 100000.0),
 ('Lax', 'Carl', 50000.0)]

In [10]:
for i in cursor.execute("Select * from employees"):
    print(i)

('Maria', 'Lay', 100000.0)
('Lax', 'Carl', 50000.0)
('Maria', 'Lay', 100000.0)
('Lax', 'Carl', 50000.0)


In [11]:
name="King"
surname = "Aut"
salary = 40000

cursor.execute("INSERT INTO employees VALUES (?,?,?)",(name, surname, salary))
cursor.execute("Select * from employees")
cursor.fetchall()

[('Maria', 'Lay', 100000.0),
 ('Lax', 'Carl', 50000.0),
 ('Maria', 'Lay', 100000.0),
 ('Lax', 'Carl', 50000.0),
 ('King', 'Aut', 40000.0)]

In [17]:
conn.close()

#### 2. Pandas df and SQL Interaction

##### 2.1 Basic import and pull between SQLite and Pandas df

In [18]:
conn = sqlite3.connect("data/test2.db")

In [ ]:
# download stock data
df = yf.download('AAPL', start='2022-01-01', end='2022-11-01')
print(df.head())
print(df.info())

[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2022-01-03  178.879944  179.734993  174.653904  174.771835  104487900
2022-01-04  176.609619  179.793905  176.039592  179.489238   99310400
2022-01-05  171.911819  177.071533  171.636636  176.521166   94537600
2022-01-06  169.042068  172.285320  168.688259  169.730027   96904000
2022-01-07  169.209137  171.145259  168.088742  169.916756   86709100
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 209 entries, 2022-01-03 to 2022-10-31
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   (Close, AAPL)   209 non-null    float64
 1   (High, AAPL)    209 non-null    float64
 2   (Low, AAPL)     209 non-null    float64
 3   (Open, AAPL)    209 non-null    float64
 4   (Volume, AAPL)  209 non-null    int64  
dt

In [22]:
# save to db
df.to_sql('AAPL', conn)

209

In [25]:
# read from sql
query = "select * from AAPL"
pd.read_sql(query, conn)

,Date,"('Close', 'AAPL')","('High', 'AAPL')","('Low', 'AAPL')","('Open', 'AAPL')","('Volume', 'AAPL')"
0,2022-01-03 00:00:00,178.879944,179.734993,174.653904,174.771835,104487900
1,2022-01-04 00:00:00,176.609619,179.793905,176.039592,179.489238,99310400
2,2022-01-05 00:00:00,171.911819,177.071533,171.636636,176.521166,94537600
3,2022-01-06 00:00:00,169.042068,172.285320,168.688259,169.730027,96904000
4,2022-01-07 00:00:00,169.209137,171.145259,168.088742,169.916756,86709100
...,...,...,...,...,...,...
204,2022-10-25 00:00:00,150.339722,150.487761,147.398854,148.119265,74732300
205,2022-10-26 00:00:00,147.388992,149.994328,146.096180,148.977853,88194300
206,2022-10-27 00:00:00,142.898697,147.092892,142.237496,146.125764,109180200
207,2022-10-28 00:00:00,153.695053,155.431938,145.879049,146.254049,164762400


##### 2.2 Adding new data

In [ ]:
# only read data newer than max date, then insert into db
max_date = pd.read_sql("select max(Date) from AAPL", conn).values[0][0] # getting max date
print(max_date)

df_new = yf.download('AAPL', start=pd.to_datetime(max_date), end="2023-01-01") # use pd.to_datetime to convert
df_new = df_new[df_new.index>max_date] # filter for 
print(df_new.head())

2022-10-31 00:00:00


[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            AAPL        AAPL        AAPL        AAPL       AAPL
Date                                                                 
2022-11-01  148.671906  153.408883  147.171875  153.043746   80379300
2022-11-02  143.125687  150.171934  143.096082  146.994213   93604600
2022-11-03  137.056473  140.925000  136.928175  140.194711   97918500
2022-11-04  136.789551  141.030238  132.835524  140.456902  140814800
2022-11-07  137.323334  137.550686  134.110687  135.534139   83374600


In [28]:
# add new data to db
df_new.to_sql("AAPL", conn, if_exists='append')

42

##### 2.3 Leverage Intraday Crypto Data

In [30]:
# switch to SGP region for codes below
client = Client()

In [31]:
df = pd.DataFrame(client.get_historical_klines(symbol='BTCUSDT',
                                          interval=Client.KLINE_INTERVAL_1MINUTE,
                                          start_str='1 d ago')) # get minite kline for the past day

def transform_df(df):

    df = df.iloc[:,:6]
    df.columns = ['time','open','high','low','close','volume']
    df['time'] = pd.to_datetime(df['time'], unit='ms', utc=True)
    df.set_index('time', inplace=True)
    df.index = df.index.tz_convert('America/New_York')

    return df

minute_df = transform_df(df)
minute_df.head()

,open,high,low,close,volume
time,,,,,
2025-02-24 13:01:00-05:00,94602.01000000,94653.27000000,94602.01000000,94645.68000000,18.15348000
2025-02-24 13:02:00-05:00,94645.69000000,94744.04000000,94645.68000000,94707.55000000,14.19364000
2025-02-24 13:03:00-05:00,94706.00000000,94706.01000000,94641.55000000,94650.01000000,7.20204000
2025-02-24 13:04:00-05:00,94650.00000000,94655.90000000,94590.18000000,94623.22000000,7.89871000
2025-02-24 13:05:00-05:00,94623.22000000,94635.98000000,94550.00000000,94596.83000000,17.00013000


In [33]:
minute_df.to_sql("BTCUSDT", conn)

1440

In [40]:
# adding new data if any
max_date = pd.read_sql("select max(time) from BTCUSDT", conn).values[0][0] # getting max date
print(max_date)

df_new = pd.DataFrame(client.get_historical_klines(symbol='BTCUSDT',
                                          interval=Client.KLINE_INTERVAL_1MINUTE,
                                          start_str='1 d ago'))
df_new = transform_df(df_new)
df_new = df_new[df_new.index>pd.to_datetime(max_date)]
print(df_new)

# only store data without the last row because incomplete
df_new[:-1].to_sql("BTCUSDT", conn, if_exists='append')
print(len(df_new[:-1]), " new rows found and inserted")

2025-02-25 13:00:00-05:00
                                     open            high             low  \
time                                                                        
2025-02-25 13:01:00-05:00  87258.96000000  87433.95000000  87258.96000000   
2025-02-25 13:02:00-05:00  87418.00000000  87432.40000000  87267.65000000   
2025-02-25 13:03:00-05:00  87297.92000000  87442.38000000  87295.20000000   
2025-02-25 13:04:00-05:00  87422.05000000  87521.74000000  87379.67000000   
2025-02-25 13:05:00-05:00  87498.33000000  87547.17000000  87407.76000000   
2025-02-25 13:06:00-05:00  87476.01000000  87672.91000000  87476.00000000   
2025-02-25 13:07:00-05:00  87667.39000000  87753.91000000  87647.68000000   

                                    close       volume  
time                                                    
2025-02-25 13:01:00-05:00  87418.01000000  26.59223000  
2025-02-25 13:02:00-05:00  87297.92000000  14.38991000  
2025-02-25 13:03:00-05:00  87422.04000000  24.544720

In [42]:
df_new

,open,high,low,close,volume
time,,,,,
2025-02-25 13:01:00-05:00,87258.96000000,87433.95000000,87258.96000000,87418.01000000,26.59223000
2025-02-25 13:02:00-05:00,87418.00000000,87432.40000000,87267.65000000,87297.92000000,14.38991000
2025-02-25 13:03:00-05:00,87297.92000000,87442.38000000,87295.20000000,87422.04000000,24.54472000
2025-02-25 13:04:00-05:00,87422.05000000,87521.74000000,87379.67000000,87498.33000000,23.55880000
2025-02-25 13:05:00-05:00,87498.33000000,87547.17000000,87407.76000000,87476.01000000,26.29298000
2025-02-25 13:06:00-05:00,87476.01000000,87672.91000000,87476.00000000,87667.38000000,29.48632000
2025-02-25 13:07:00-05:00,87667.39000000,87753.91000000,87647.68000000,87753.91000000,18.84061000


#### 3. Putting it altogether as functions

##### 3.1 for Stock Data (from yfinance)

In [49]:
# for yf stock data
def stock_sql_handler(symbol, start = '2022-01-01'):
    try: # if the table exists:
        max_date = pd.read_sql(f"select max(Date) from {symbol}", conn).values[0][0]
        print(max_date)

        new_rows = yf.download(symbol, start=pd.to_datetime(max_date))
        new_rows = new_rows[new_rows.index>max_date]
        new_rows.to_sql(symbol, conn, if_exists='append')
        print(str(len(new_rows)) + " new rows imported to db")
    except: # if no such table
        new_data = yf.download(symbol, start=start)
        new_data.to_sql(symbol, conn)
        print(f"New data created for {symbol} with {len(new_data)} rows")


In [50]:
stock_sql_handler('TSLA')

[*********************100%***********************]  1 of 1 completed

New data created for TSLA with 789 rows


In [51]:
stocks = ['GOOG','TSLA','AAPL','MMM']
for s in stocks:
    stock_sql_handler(s)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


New data created for GOOG with 789 rows
2025-02-25 00:00:00
0 new rows imported to db
2025-02-25 00:00:00


[*********************100%***********************]  1 of 1 completed


0 new rows imported to db


[*********************100%***********************]  1 of 1 completed

New data created for MMM with 789 rows


##### 3.2 for Crypto Data (from Binance)

In [52]:
# switch to SGP region for codes below
client = Client()

In [63]:
def crypto_sql_handler(symbol, start = "1 day ago"):
    try: # if table exists
        max_date = pd.read_sql(f"select max(time) from {symbol}", conn).values[0][0] # getting max date
        print(max_date)

        df_new = pd.DataFrame(client.get_historical_klines(symbol=symbol,
                                          interval=Client.KLINE_INTERVAL_1MINUTE,
                                          start_str=str(max_date)))
        df_new = transform_df(df_new)
        df_new = df_new[df_new.index>pd.to_datetime(max_date)]

        # only store data without the last row because incomplete
        df_new[:-1].to_sql(symbol, conn, if_exists='append')
        print(len(df_new[:-1]), " new rows found and inserted")
    except: # if no such table
        new_data = pd.DataFrame(client.get_historical_klines(symbol=symbol,
                                          interval=Client.KLINE_INTERVAL_1MINUTE,
                                          start_str = start))
        new_data = transform_df(new_data)

        # only store data without the last row because incomplete
        new_data[:-1].to_sql(symbol, conn)
        print(f"New data created for {symbol}, imported {len(new_data[:-1])} rows")

In [64]:
crypto_sql_handler('BTCUSDT')

2025-02-25 13:45:00-05:00
0  new rows found and inserted


In [65]:
cryptos = ['ETHUSDT','SOLUSDT','XRPUSDT','BTCUSDT']
for c in cryptos:
    crypto_sql_handler(c)

2025-02-25 13:44:00-05:00
1  new rows found and inserted
2025-02-25 13:44:00-05:00
1  new rows found and inserted
2025-02-25 13:44:00-05:00
1  new rows found and inserted
2025-02-25 13:45:00-05:00
0  new rows found and inserted
